# Thanks Recipient Experiment Power Analysis
[J. Nathan Matias](https://github.com/natematias)
February 2019

Some components of this are drawn from [github.com/natematias/poweranalysis-onlinebehavior](https://github.com/natematias/poweranalysis-onlinebehavior).

Eventually, this power analysis code will ask a series of questions of [historical data prepared by Max Klein](https://docs.google.com/document/d/1RKJZqoWKQuWDoKk94drIEsJWK6kBUeZ8KIJOyEqDTTE/edit) and produce a series of answers used for power analysis and study design in CivilServant's research with Wikipedians on [the effects of giving thanks to other Wikipedians](https://meta.wikimedia.org/wiki/Research:Testing_capacity_of_expressions_of_gratitude_to_enhance_experience_and_motivation_of_editors):
* The experiment plan is on Overleaf: [Experiment Plan: Mentoring and Protection in Wikipedia Moderation](https://www.overleaf.com/project/5c379e06f882d02f5b8c9f44)

This analysis will define and report the following:

* Assumptions about minimum observable treatment effects for each DV
* Reports on the statistical power, bias, and type S error rate for all possible estimators, given the above assumptions
* Data-driven decisions:
    * Decisions about the final set of measures to use
    * Decisions about the randomization procedure
    * Decisions about the final estimators to use
    * Decisions about the sample size to specify for the experiment
    * Decisions about any stop rules to use in the experiment

**Note:** Since the thanks recipient study will involve participants on multiple language Wikipedias, this code defines a procedure that can be reproduced for the following language wikipedias:
* German
* Persian
* Arabic
* Plish

# Load Libraries

In [77]:
options("scipen"=9, "digits"=4)
library(dplyr)
library(MASS)
library(ggplot2)
library(rlang)
library(tidyverse)
library(viridis)
library(DeclareDesign)
## Installed DeclareDesign 0.13 using the following command:
# install.packages("DeclareDesign", dependencies = TRUE,
#                 repos = c("http://R.declaredesign.org", "https://cloud.r-project.org"))

## DOCUMENTATION AT: https://cran.r-project.org/web/packages/DeclareDesign/DeclareDesign.pdf
cbPalette <- c("#999999", "#E69F00", "#56B4E9", "#009E73", "#F0E442", "#0072B2", "#D55E00", "#CC79A7")
options(repr.plot.width=7, repr.plot.height=3.5)
sessionInfo()

R version 3.5.1 (2018-07-02)
Platform: x86_64-pc-linux-gnu (64-bit)
Running under: Ubuntu 16.04.5 LTS

Matrix products: default
BLAS: /usr/lib/libblas/libblas.so.3.6.0
LAPACK: /usr/lib/lapack/liblapack.so.3.6.0

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
 [1] DeclareDesign_0.12.0 estimatr_0.14        fabricatr_0.6.0     
 [4] randomizr_0.16.1     viridis_0.5.1        viridisLite_0.3.0   
 [7] forcats_0.3.0        stringr_1.3.1        purrr_0.2.5         
[10] readr_1.2.1          tidyr_0.8.2          tibble_1.4.2        
[13] tidyverse_1.2.1      rla

# Utility Methods

In [2]:
# Return the difference in mu associated with an incidence rate ratio
# from a negative binomial model. This difference can then be used to
# simulate a negative binomial distribution for the effect of a given irr
#                                                                       
#` @param mu The baseline mu in question                               
#` @param irr The incidence rate ratio in question
mu.diff.from.mu.irr <- function(mu, irr){
    mu*(irr-1)
}

# Return the total sum of betas for a
# logistic regression, given a probability
#
#` @param p the probability in question
betas.logit.from.prob <- function(p){
    log(p/(1-p))
}


# Return the total sum of betas for a
# logistic regression, given a probability
#
#` @param Y list of observed Ys
betas.logit.from.mean <- function(Y){
    p = mean(Y)
    log(p/(1-p))
}

# Return the minimum power reported in a diagnosis
# 
#` @param diagnosis
min.diagnosis.power <- function(diagnosis){
    min(diagnosis$diagnosands_df['power'])
}

In [3]:
# Conduct a binary search for a certain level of statistical power
# within the constraints of a configuration file
#
#` @param config.df The configuration file in question
#` @diagnosis.method The method that conducts a single DeclareDesign diagnosis and returns the diagnosis
#` @target.power The statistical power that ideally should be the minimum across the study
#` @target.tolerance How close to the desired statistical power is close enough?
#` @min.sample.diff If the search is close enough that the change is less than min.sample.diff, end the search
#` @start.sample.size if specified, use the starting value as the initial sample size to use

search.for.power <- function(config.df, diagnosis.method = diagnose.experiment, 
                             target.power = 0.85, target.tolerance = 0.01, 
                             min.sample.diff = 100,
                             start.sample.size = NA){  
    max.sample.size = config.df$n.max
    min.sample.size = config.df$n.min
    if(is.na(start.sample.size)){
        current.sample.size = as.integer(max.sample.size / 2)
    }else{
        current.sample.size = start.sample.size
    }
    current.power = 0.0

    ## Initialize first iteration
    print(paste("min:", min.sample.size, "max:", max.sample.size, "current:", current.sample.size))
    flush.console()

    ptm = proc.time() #record time the simulation started
    ddf <- diagnosis.method(current.sample.size, config.df)
    ddf$diagnosands$n <- current.sample.size
    diagnoses.df = ddf$diagnosands
    current.power <- min.diagnosis.power(ddf)

    ## output elapsed time for this iteration
    time.elapsed <- proc.time() -  ptm
    print(paste("     seconds:", as.integer(time.elapsed['elapsed'])))

    
    if(current.power < target.power){
        min.sample.size = current.sample.size
        print(paste(current.power, "<", target.power))
    }else{
        max.sample.size = current.sample.size
        print(paste(current.power, ">", target.power))
    }

    current.sample.size = min.sample.size + as.integer((max.sample.size - min.sample.size)/2)
    while(all.equal(target.power, current.power, tolerance = target.tolerance)!=TRUE){
        print(paste("min:", min.sample.size, "max:", max.sample.size, "current:", current.sample.size))
        flush.console()

        ptm = proc.time() #record time the simulation started

        ## conduct simulations
        ddf <- diagnosis.method(current.sample.size, config.df)
        ddf$diagnosands$n <- current.sample.size
        ## append simulation results to dataframe
        diagnoses.df <- rbind(diagnoses.df, ddf$diagnosands)
        
        ## output elapsed time for this iteration
        time.elapsed <- proc.time() -  ptm
        print(paste("     seconds:", as.integer(time.elapsed['elapsed'])))


        ## check the current statistical power and
        ## carry out the binary search by first updating the boundaries
        current.power <- min.diagnosis.power(ddf)
        if(current.power < target.power){
            min.sample.size = current.sample.size
            print(paste(current.power, "<", target.power))
        }else{
            max.sample.size = current.sample.size
            print(paste(current.power, ">", target.power))
        }
        ## update the current pointer, or break if
        ## the sample size difference is less than or equal
        ## to ten
        sample.size.diff <- as.integer((max.sample.size - min.sample.size)/2)
        if(abs(sample.size.diff) <= min.sample.diff){
            print(paste("Sample size difference ", abs(sample.size.diff), 
                        " <= ", min.sample.diff, ". Ending cycle.", sep=""))
            break
        }
        current.sample.size = min.sample.size + sample.size.diff
    }
    diagnoses.df
}

In [4]:
# Iterate linearly for a certain level of statistical power
# within the constraints of a configuration file
# at a certain sample size increment. Useful for
# illustrating ideas, or for comparing estimators with
# very different statistical power, where the binary search
# will optimize for the worst estimator but not show useful
# indormation about more efficient estimators
#
#` @param config.df The configuration file in question
#` @diagnosis.method The method that conducts a single DeclareDesign diagnosis and returns the diagnosis
#` @iteration.interval when iterating, use this interval between sample sizes

iterate.for.power <- function(config.df, diagnosis.method = diagnose.experiment, 
                             iteration.interval){  
    max.sample.size = config.df$n.max
    min.sample.size = config.df$n.min
    current.sample.size = min.sample.size
    
    iteration.count = ceiling((max.sample.size - min.sample.size) / iteration.interval)

    ## Initialize first iteration
    print(paste("min:", min.sample.size, "max:", max.sample.size, "current:", current.sample.size))
    flush.console()

    ptm = proc.time()
    ddf <- diagnosis.method(current.sample.size, config.df)
    ddf$diagnosands$n <- current.sample.size
    diagnoses.df = ddf$diagnosands
    current.power <- min.diagnosis.power(ddf)
    time.elapsed <- proc.time() -  ptm
    print(paste("     seconds:", as.integer(time.elapsed['elapsed'])))
    
    for(i in seq(1, iteration.count)){
        current.sample.size = current.sample.size + iteration.interval
        print(paste("min:", min.sample.size, "max:", max.sample.size, "current:", current.sample.size))
        flush.console()
    
        ptm = proc.time()
        ## conduct simulations
        ddf <- diagnosis.method(current.sample.size, config.df)
        ddf$diagnosands$n <- current.sample.size
        ## append simulation results to dataframe
        diagnoses.df <- rbind(diagnoses.df, ddf$diagnosands)
        time.elapsed <- proc.time() -  ptm
        print(paste("     seconds:", as.integer(time.elapsed['elapsed'])))
    }
    diagnoses.df
}

In [5]:
# Create a plot of a power search or iteration output
# Especially useful in cases with multiple DVs or estimators
#
#' @param diagnoses Dataframe of diagnosis info
#` @param config.df the power analysis config dataframe

plot.power.results <- function(diagnoses, config.df){
    for(estimator_label in unique(diagnoses$estimator_label)){
        estimator.diagnoses <- diagnoses[diagnoses$estimator_label==estimator_label,]
        estimator_min_sample = min(estimator.diagnoses$n[estimator.diagnoses$power>0.8])
        p <- ggplot(data=estimator.diagnoses, aes(n, power)) +
                geom_point(color="coral") +
                xlab("sample size") +
                ylim(0,1) +
                geom_hline(aes(yintercept=0.8), linetype="dashed") +
                theme_light() +
                ggtitle(paste(config.df$pa.label, ": statistical Power for Estimator ", estimator_label, "\n",
                              "Minimum sample: ", estimator_min_sample, sep="")) +
                ggsave(paste("figures/power.analysis.", make.names(estimator_label), ".", config.df$pa.label, ".png", sep=""))
    }
}

# Load Power Analysis Dataframes and Review The Data

In [13]:
data.path <- "~/Tresors/CivilServant/projects/wikipedia-integration/gratitude-study/datasets/power_analysis"
de.power.df <- read.csv(file.path(data.path, "de_gratitude_power-analysis_dataset_sim_date_20180306_v1.csv"))
fa.ar.pl.power.df <- de.power.df <- read.csv(file.path(data.path, "gratitude_power-analysis_dataset_sim_date_20180306_v1.csv"))
fa.power.df <- subset(fa.ar.pl.power.df, lang="fa")
ar.power.df <- subset(fa.ar.pl.power.df, lang="ar")
pl.power.df <- subset(subset(fa.ar.pl.power.df, lang="fa"))

simulated.treatment.date <- as.Date("20180306", "%Y%M%D")

## Issues with the dataset
This is a list of discrepancies between the specification and the data file, Feb 2019
* `num_prev_thanks_before_treatment` should be called `num_prev_thanks_pre_treatment`, as specified in the "data needed" file
* `newcomer` should be defined in the CSV
* `labor` should be spelled consistently

## Review Blocking & Assignment Variables

In [62]:
# VARIABLE: experience_level_pre_treatment: Definition: 
#  the elapsed number of days between registration 
#  and last edit in the observation period up to the simulated treatment date

de.power.df$prev_experience <- as.integer(gsub("bin_", "", de.power.df$experience_level_pre_treatment))
de.power.df$prev_experience <- factor(de.power.df$prev_experience, 
                                      levels = sort(unique(de.power.df$prev_experience)))

summary(factor(de.power.df$prev_experience))

# VARIABLE: newcomer: accounts that were created within in the last 90 days, 
#   and which have made at least 3 edits since starting their account
de.power.df$newcomer <- de.power.df$experience_level_pre_treatment == "bin_0" & 
    de.power.df$num_edits_90_pre_treatment >= 3

# VARIABLE: num_prev_thanks_pre_treatment
de.power.df$num_prev_thanks_pre_treatment <- de.power.df$num_prev_thanks_before_treatment

0   90  180  365  730 1460 2920 
1387 1200 1200 1200 1200 1200 1200

Notice that active Wikipedia editors who have been around for longer and who are part of this sample tend to write more edits and put in more labor hours on average. This is a *very* good reason to block on those characteristics.

In [64]:
## SHOW NUM EDITS BY EXPERIENCE GROUP:
aggregate(de.power.df[c("num_edits_90_pre_treatment")],
          FUN=mean, by = list(de.power.df$prev_experience))

## SHOW LABOR HOURS BY EXPERIENCE GROUP:
aggregate(de.power.df[c("labour_hours_90_pre_treatment")],
          FUN=mean, by = list(de.power.df$prev_experience))

Group.1,num_edits_90_pre_treatment
0,3.969
90,13.292
180,41.049
365,45.230
730,80.653
1460,74.476
2920,98.986


Group.1,labour_hours_90_pre_treatment
0,1.865
90,3.959
180,4.960
365,6.633
730,7.893
1460,9.635
2920,14.685


Notice that the number of previous thanks are strongly correlated with the experience groups. This will simplify our paired matching based on Mahalanobis distance.

In [65]:
## SHOW PREVIOUS THANKS BY EXPERIENCE GROUP:
aggregate(de.power.df[c("num_prev_thanks_pre_treatment")],
          FUN=mean, by = list(de.power.df$prev_experience))

Group.1,num_prev_thanks_pre_treatment
0,0.04182
90,0.30583
180,0.71250
365,1.63667
730,3.22750
1460,10.43250
2920,19.13167


## Construct and Review Behavioral Dependent Variables

In [72]:
# VARIABLE: num.edit.diff: the number of edits
# in the 90 days after treatment minutes the number in the 90 before treatment
de.power.df$num.edit.diff <- de.power.df$num_edits_90_post_treatment - 
    de.power.df$num_edits_90_pre_treatment

# VARIABLE: num.edit.diff: the number of edits
# in the 90 days after treatment minutes the number in the 90 before treatment
de.power.df$labor.hour.diff <- de.power.df$labour_hours_90_post_treatment - 
    de.power.df$labour_hours_90_pre_treatment

Here we see that the average number of edits 90 days before and after imagined treatment is declining for every group except the groups that have been around for 730 to 1460 days.

In [75]:
## SHOW NUM EDIT DIFF BY EXPERIENCE GROUP:
aggregate(de.power.df[c("num.edit.diff")],
          FUN=mean, by = list(de.power.df$prev_experience))

Group.1,num.edit.diff
0,-1.657
90,-5.921
180,-5.971
365,-5.104
730,-12.426
1460,14.837
2920,-15.012


Here we see that the average number of labor hours 90 days before and after is declining for every group.

In [76]:
## SHOW LABOR HOUR DIFF BY EXPERIENCE GROUP:
aggregate(de.power.df[c("labor.hour.diff")],
          FUN=mean, by = list(de.power.df$prev_experience))

Group.1,labor.hour.diff
0,-1.1435
90,-2.0424
180,-0.9263
365,-1.2137
730,-1.3326
1460,-0.1311
2920,-1.7913


## Construct and Review a Person-Period Dataset for Survival Analysis